<a href="https://colab.research.google.com/github/passivebook/FixWordPressLinks/blob/main/Fix_WordPress_Links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive/folders/1T5JJmlDwdaEkAAbpH2VOn-TCeAuW8OKl

In [1]:
import os
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


#### Put the path to the google drive folder after `/content/gdrive/My Drive/`



In [2]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Fix WordPress Links")

#### Upload the post.txt file to Google Drive 
1. Create a `post.txt` file and paste the WordPress Post Code in it. 
2. Upload the post.txt file to path setup above. 

### Check file post.txt exists in Drive by running the code below

In [3]:
! ls

'Fix WordPress Links.ipynb'   input.txt   output.txt


### File Name Uploaded

In [4]:
filename = 'input.txt'
domain = 'passivebook.com'
affiliate_link = 'passivebook.com/go'

In [9]:
html_doc = """
<!-- wp:paragraph -->
<p><a class="test" href="https://passivebook.com/go/trello"> Affiliate Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://passivebook.com/calculators/blog-earning-calculator/" class="test">Internal Link </a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://google.com">External Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="#jumplink">Jump Link</a></p>
<!-- /wp:paragraph -->

<!-- wp:paragraph -->
<p><a href="https://passivebook.com/go/TRELLO.md">Markdown</a></p>
<!-- /wp:paragraph -->
"""

soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
f = open(filename)
content = f.read()
#parse HTML
soup = BeautifulSoup(content, 'html.parser')

In [11]:
# Operations on All Links
for link in soup.find_all('a'):
    # Remove .md Obsidian Markdown from links
    link['href'] = link['href'].replace(".md", "")
    # Check if link is not an anchor link
    if "#" not in link['href']:
      # Set All Links to Open in New Tab
      link['target'] = '_blank'
      # Remove Leading and Trailiing White Spaces
      link.string = link.string.strip()

# Internal Links
for link in soup.find_all('a', href=re.compile(domain)):
    # Convert all internal links to lower case
    link['href'] = link['href'].lower()        
    url = link['href']
    # Add Trailing / to all links
    if url[len(url)-1] != "/" and "#" not in link['href']:
        link['href'] = url + "/"

# Affiliate Links
for link in soup.find_all('a', href=re.compile(affiliate_link)):
    # Make Affiliate Links No Follow
    link['rel'] = 'nofollow'

# External Links
#for link in soup.find_all('a', href=re.compile(".*?:\\/\\/(?!"+domain+").*?")):
    #link['rel'] = link['rel'].remove('nofollow')
    #link['rel'] = 'nofollow'

print(soup)


<!-- wp:paragraph -->
<p><a class="test" href="https://passivebook.com/go/trello/" rel="nofollow" target="_blank">Affiliate Link</a></p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p><a class="test" href="https://passivebook.com/calculators/blog-earning-calculator/" target="_blank">Internal Link</a></p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p><a href="https://google.com" target="_blank">External Link</a></p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p><a href="#jumplink">Jump Link</a></p>
<!-- /wp:paragraph -->
<!-- wp:paragraph -->
<p><a href="https://passivebook.com/go/trello/" rel="nofollow" target="_blank">Markdown</a></p>
<!-- /wp:paragraph -->



In [ ]:
with open("output.txt", "w", encoding='utf-8') as file:
    file.write(str(soup))